<a href="https://colab.research.google.com/github/morozoff-dev/myTgBot/blob/main/myBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# prog:


In [ ]:
import random

In [ ]:
text = input()
if text in ["Привет", "Здарова", "Хеллоу"]:
  print(random.choice(["Здрасте", "Йоу", "Приветики"]))
elif text in ["Пока", "Увидимся", "Чао"]:
  print(random.choice(["Буду ждать нашей встречи", "Ок", "Бай-бай"]))
else:
  print("Не понял")


In [ ]:
import re
import nltk
# filter_text - фильтрует текст
# "Привет" => "привет"
# "привет !!! :)" => "привет"
# "  привет  " => "привет"
def filter_text(text):
    text = text.lower()  # lower - к нижнему регистру
    text = text.strip()  # strip - вырезать пробелы с начала и конца строки
    pattern = r"[^\w\s]" # Все что не слово и не пробел
    text = re.sub(pattern, "", text) # Из переменной text вырезаем "Все что не слово и не пробел"
    return text

# True, если тексты похожи (user_text пользовательский текст, example - контрольная фраза)
# False, если не похожи
def text_match(user_text, example):
    user_text = filter_text(user_text) # Отфильтруем лишнее из первой строки
    example = filter_text(example) # Отфильтруем лишнее из  строки
    if user_text == example: # Если тексты точно совпадают
        return True
    if user_text.find(example) != -1: # Если фраза входит в пользовательский текст
        return True

    distance = nltk.edit_distance(user_text, example)
    # Отношение кол-ва ошибок к длине слова
    ratio = distance / len(example)
    if ratio < 0.40:
        return True

    return False

In [ ]:
# Тексты совпадают
text_match("Привет", "Привет")

True

In [ ]:
# Разный регистр
text_match("привет", "Привет")

True

In [ ]:
# Лишние символы
text_match("Привет!!!", "Привет")

True

In [ ]:
# Пробелы
text_match("   Привет!!!   ", "Привет")

True

In [ ]:
# Лишние слова
text_match("чувачок Привет, какие твои делища", "Привет")

True

In [ ]:
# Опечатки
text_match("Превет", "Привет")


True

# Определение "намерения" пользователя

In [ ]:
# Составим карты "намерений", которые поддерживает наш бот
INTENTS = {
    "hello": {
        "examples": ["Привет", "Здравствуйте", "Добрый день"],
        "responses": ["Йоу", "Здарова", "Приветствую тебя, человек."],
    },
    "bye": {
        "examples": ["Пока", "До свидания", "Всего хорошего"],
        "responses": ["Давайдосвиданья", "И вам приятного денечка"],
    },
    "how_are_you": {
        "examples": ["Как дела"],
        "responses": ["Функционирую нормально"],
    },
    "buy_some_pizza": {
        "examples": ["Голоден", "Хочу есть", "Пицца"],
        "responses": ["А вот фиг тебе"],
    },
}

In [ ]:
import random
# Функция которая находит намерение пользователя по его тексту (с помощью text_match)
def get_intent(user_text):
    # Для каждого интента в карте, сделать следующее:
    for intent in INTENTS:
        examples = INTENTS[intent]["examples"]
        for example in examples:
            if len(filter_text(example)) < 3:
                continue
            if text_match(user_text, example):
                return intent # Найденное намерение подходит к польз. тексту
    return None # Ничего не найдено

# Функция возвращает случайную фразу ответа по данном интенту
def get_random_response(intent):
    return random.choice(INTENTS[intent]["responses"])

# ML


# Классификация текстов ML-моделью
Файл с данными: https://drive.google.com/file/d/1_L5CYGsO58zkB3LMBG73ezIEwYFD07Ed/view

In [ ]:
import json

# Взять файл и сделать с ним следующее:
with open("/content/big_bot_config.json", "r") as config_file:
    data = json.load(config_file)

INTENTS = data["intents"]

In [ ]:
print("Интентов загружено из файла:", len(INTENTS))

Интентов загружено из файла: 439


In [ ]:
# Фраза (X) - на вход
# Интент (y) - на выход
X = []
y = []

for intent in INTENTS:
    examples = INTENTS[intent]["examples"]
    for example in examples:
        example = filter_text(example)
        if len(example) < 3:
            continue # Пропускаем текст если он слишком короткий
        X.append(example)  # Добавляем фразу в список X
        y.append(intent)

# Векторизация


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

vectorizer.fit(X) # Обучаем векторайзер

vecX = vectorizer.transform(X) # Все тексты преобразуем в вектора

In [ ]:
# Обучаем модель классификации
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(vecX, y) # Обучение модели

RandomForestClassifier()

In [ ]:
# Метрика
from sklearn.metrics import accuracy_score, f1_score
# Интенты предсказанные моделью
y_pred = model.predict(vecX)

print("accuracy_score", accuracy_score(y, y_pred)) # Сравниваем y и y_pred
print("f1_score", f1_score(y, y_pred, average="macro")) # Сравниваем y и y_pred

accuracy_score 0.829137294394092
f1_score 0.7562980674891996


In [ ]:
def get_intent_ml(user_text):
    user_text = filter_text(user_text)
    vec_text = vectorizer.transform([user_text])
    intent = model.predict(vec_text)[0]
    # model.predict_proba()
    return intent

In [ ]:
import pandas as pd
proba = model.predict_proba(vectorizer.transform(["Привет, как дела"]))

# Вывести на экран список вероятных интентов
pd.DataFrame(columns=model.classes_, data=[proba[0]]).T.sort_values(by=0, ascending=False)

,0
hello,0.392639
howareyou,0.040721
mood,0.036090
howAreYou,0.033672
howdoyoudo,0.033637
...,...
fables,0.000000
experience,0.000000
exits,0.000000
exams_and_quizes,0.000000


In [ ]:
def bot(user_text):
    intent = get_intent(user_text)
    if intent:
        return get_random_response(intent)
    intent = get_intent_ml(user_text)
    return get_random_response(intent)

# Telegram


In [ ]:
! pip install python-telegram-bot --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
TOKEN="введите ваш токен"

In [ ]:
from telegram import Update # Обновление пришедшее к нам с серверов ТГ
from telegram.ext import ApplicationBuilder, MessageHandler, filters

# Создаем и настраиваем бот-приложение
app = ApplicationBuilder().token(TOKEN).build()

async def telegram_reply(upd: Update, ctx):
    name = upd.message.from_user.full_name
    user_text = upd.message.text
    print(f"{name}: {user_text}")
    reply = bot(user_text)
    print(f"BOT: {reply}")
    await upd.message.reply_text(reply)

handler = MessageHandler(filters.TEXT, telegram_reply) # Создаем обработчик текстовых сообщений
app.add_handler(handler) # Добавляем обработчик в приложение

# app.run_polling # Наш код регулярно опрашивает сервер на предмет новых Апдейтов
# app.run_webhook # Запускает веб-сервер, к которому будет подключаться сам ТГ и присылать туда апдейты
app.run_polling()